# Ensemble Kalman Filter
This demonstrates the Ensemble Kalman filter for the discretized pendulum $x'' = -\sin(x)$.  It is an analog on example 7.2 from Sullivan.

In [ ]:
using Plots
using Random
using Distributions
using Statistics
using Printf
using LinearAlgebra
using LaTeXStrings

In [ ]:
default(xtickfont=font(12),  ytickfont=font(12), guidefont=font(12), 
    legendfont=font(10), lw=2,ms=4)

In [ ]:
Δt = 0.1;
γ = sqrt(.5);
f = X-> [X[1] + Δt * X[2] - Δt^2 * sin(X[1]); X[2]-Δt * sin(X[1])];

H = Float64[1 0];
R = [γ^2];

# true initial condition
Xt_0 = Float64[0; 1];

E = 10^2; # size of ensemble

n = 10^2; # number of steps

# generate data
Xt_path = [copy(Xt_0)];
Random.seed!(500);
y_data = Array{Float64}[];
Xt = copy(Xt_0);
for j in 1:n
    Xt = f(Xt);
    y = H * Xt .+ rand(Normal(0,γ));
    push!(Xt_path, copy(Xt));
    push!(y_data, y);
end


In [ ]:
# set initial condition
Random.seed!(1000)
X0 = randn(2, E)

In [ ]:
plot(0:n, [X_[1] for X_ in Xt_path],label=L"$x_n$", legend=:bottomright)
scatter!(1:n, [y_[1] for y_ in y_data], label=L"$y_n$")
scatter!(0*ones(E), [x_ for x_ in X0[1,:]], label="Starting Ensemble")
xlabel!(L"$n$")

In [ ]:
X = copy(X0);
# extract means and covariances
XE = mean(X0, dims =2);
CE = cov(X0, dims=2);
X_path = [deepcopy(XE)];
C_path = [deepcopy(CE)];
D = zeros(1, E);
    
Random.seed!(5000);
for i in 1:n
    # predict
    for e in 1:E
        X[:,e] .= f(X[:,e]);
    end
    # empirical covariance after prediction
    CE = cov(X, dims=2);
    # data replication
    for e in 1:E
        D[:,e] = y_data[i] + rand(Normal(0,γ), 1,1);
    end
    # correct
    K = CE * H' * inv(H * CE * H' + R)
    for e in 1:E
        X[:,e] = X[:,e] + K * (D[:,e] - H * X[:,e]);
    end
    # compute mean and covariance after correction for analysis
    XE = mean(X, dims =2);
    CE = cov(X, dims=2);
    
    push!(X_path, deepcopy(XE));
    push!(C_path, deepcopy(CE));
end

In [ ]:
plot(0:n, [X_[1] for X_ in X_path], ribbon = sqrt.([C_[1,1] for C_ in C_path]),label=L"$\hat{x}_n$")
plot!(0:n, [X_[1] for X_ in Xt_path],label=L"$x_n$")
scatter!(1:n, [y_[1] for y_ in y_data],label=L"$y_n$")
ylims!(-2.5,2.5)
xlabel!(L"$n$")

In [ ]:
plot(0:n, [X_[2] for X_ in X_path], ribbon = sqrt.([C_[2,2] for C_ in C_path]),label=L"$\hat{v}_n$")
plot!(0:n, [X_[2] for X_ in Xt_path], label=L"$v_n$")
ylims!(-2.5,2.5)
xlabel!(L"$n$")